In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def clean_text(text):
    """Nettoie le texte en enlevant les espaces et caractères spéciaux superflus"""
    if text:
        # Enlève les espaces multiples et les sauts de ligne
        text = ' '.join(text.strip().split())
        # Enlève les guillemets au début et à la fin
        text = text.strip('"')
        return text
    return ''

def extract_question_number_and_text(text):
    """Extrait le numéro et le texte de la question"""
    pattern = r'(\d+)\.\s*(.+)'
    match = re.match(pattern, text)
    if match:
        return match.group(2)  # Retourne seulement le texte de la question
    return text

def scrape_greatlearning_ml_questions():
    url = "https://www.mygreatlearning.com/blog/machine-learning-interview-questions/"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        qa_pairs = []
        
        # Rechercher tous les h3 qui contiennent des questions
        questions = soup.find_all('h3', class_='wp-block-heading')
        
        for question_elem in questions:
            try:
                # Extraire le texte de la question depuis la balise strong
                strong_tag = question_elem.find('strong')
                if not strong_tag:
                    continue
                    
                question_text = clean_text(strong_tag.get_text())
                if not question_text:
                    continue
                
                # Nettoyer le texte de la question
                question_text = extract_question_number_and_text(question_text)
                
                # Trouver le paragraphe suivant qui contient la réponse
                answer_text = ""
                next_p = question_elem.find_next('p')
                
                if next_p:
                    answer_text = clean_text(next_p.get_text())
                
                # Ajouter la paire uniquement si on a une question et une réponse
                if question_text and answer_text:
                    qa_pairs.append({
                        'question': question_text,
                        'answer': answer_text
                    })
                
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une paire Q&A: {e}")
                continue
        
        # Créer un DataFrame
        df = pd.DataFrame(qa_pairs)
        
        # Sauvegarder en CSV
        df.to_csv('greatlearning_ml_questions.csv', index=False, encoding='utf-8')
        
        print(f"Scraping terminé. {len(qa_pairs)} questions-réponses ont été sauvegardées dans 'greatlearning_ml_questions.csv'")
        print("\nExemple des premières questions et réponses :")
        pd.set_option('display.max_colwidth', None)
        print(df.head())
        
    except requests.RequestException as e:
        print(f"Erreur lors de la requête: {e}")
    except Exception as e:
        print(f"Une erreur est survenue: {e}")

if __name__ == "__main__":
    scrape_greatlearning_ml_questions()

Scraping terminé. 145 questions-réponses ont été sauvegardées dans 'greatlearning_ml_questions.csv'

Exemple des premières questions et réponses :
                                                                                                                               question  \
0                                              Explain the terms Artificial Intelligence (AI), Machine Learning (ML) and Deep Learning?   
1                                                                      What are the different types of Learning/ Training models in ML?   
2                                                 What is the main key difference between supervised and unsupervised machine learning?   
3  There are many machine learning algorithms till now. If given a data set, how can one determine which algorithm to be used for that?   
4                                                                              State the differences between causality and correlation?   

                  